In [ ]:
# Uninstall everything that might conflict
!pip uninstall -y tensorflow tf-nightly keras-nightly tensorflow-datasets

# Install a compatible version
!pip install tensorflow==2.20.0 tensorflow-datasets==4.9.9


In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)  # Should show 2.20.0


In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

In [ ]:
import pandas as pd

# Read TSV with proper column names
df = pd.read_csv('train-data.tsv', sep='\t', names=['label', 'message'], on_bad_lines='skip')

# Check the first few rows
print(df.head())
print(df.shape)


In [ ]:
import pandas as pd

# Load valid dataset with proper column names
valid_df = pd.read_csv('valid-data.tsv', sep='\t', names=['label','message'], on_bad_lines='skip')

print(valid_df.head())
print("Shape:", valid_df.shape)
print("Labels distribution:\n", valid_df['label'].value_counts())


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tdf=TfidfVectorizer()
x_train=tdf.fit_transform(df['message'])
x_valid=tdf.transform(valid_df['message'])


In [ ]:
y_train=df['label'].map({'ham':0,'spam':1})
y_valid=valid_df['label'].map({'ham':0,'spam':1})

In [ ]:
from sklearn.linear_model import LogisticRegression
model=LogisticRegression(max_iter=1000)
model.fit(x_train,y_train)

In [ ]:
from sklearn.metrics import accuracy_score, classification_report
pred=model.predict(x_valid)
print('acc:',accuracy_score(y_valid,pred))
print('acc:',classification_report(y_valid,pred))

In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])

# Function to predict a single message
def predict_message(pred_text):
    # 1. Transform the text using the same TF-IDF vectorizer
    X = tdf.transform([pred_text])

    # 2. Predict probability of spam (1)
    prob_spam = model.predict_proba(X)[0][1]  # probability for class 'spam'

    # 3. Predict class label
    label_num = model.predict(X)[0]
    label = 'spam' if label_num == 1 else 'ham'

    # 4. Return list: [probability_spam, label]
    return [prob_spam, label]

# Example usage
pred_text = "how are you doing today?"
prediction = predict_message(pred_text)
print(prediction)


In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()
